In [1]:
# import module
from rdd import rdd
from scipy.stats import binned_statistic
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import data as dataframe
data = pd.read_csv("C:/Users/張翔竣/Desktop/US_election.csv")
demvoteshare = data.demvoteshare
win_dem_t1 = data.win_dem_t1
win_dem = data.win_dem
threshold = 0.5
df = pd.DataFrame({'win_dem_t1':win_dem_t1, 
                   'demvoteshare': demvoteshare,
                   'win_dem': win_dem})

In [2]:
# model 1 (graph omitted, supplied by STATA)
model = smf.ols(formula = 'win_dem_t1 ~ win_dem',data = df)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:             win_dem_t1   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                 1.628e+04
Date:                Sat, 22 Jan 2022   Prob (F-statistic):               0.00
Time:                        21:56:30   Log-Likelihood:                -1886.2
No. Observations:                9345   AIC:                             3776.
Df Residuals:                    9343   BIC:                             3791.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1287      0.005     26.898      0.0

In [5]:
# truncate the dataframe by optimal bandwidth, average the data with sample average within bin
bandwidth_opt = rdd.optimal_bandwidth(df['win_dem_t1'], df['demvoteshare'], cut=threshold)  # 0.07559328871095955
df_cutleft = df[df['demvoteshare'] >= 0.5 - bandwidth_opt]
df_cutright = df_cutleft[df_cutleft['demvoteshare'] <= 0.5 + bandwidth_opt]
df_bin = rdd.bin_data(df_cutright, 'win_dem_t1', 'demvoteshare', 300)
demvoteshare_bin = df_bin.demvoteshare
win_dem_t1_bin = df_bin.win_dem_t1
df_bin = pd.DataFrame({'y':win_dem_t1_bin, 
                       'x': demvoteshare_bin, 
                      'x2': demvoteshare_bin.pow(2)})
df_bin['Group'] = np.where(df_bin.x >= 0.5, "Treatment", "Control")

# model 2: result (graph omitted, supplied by STATA)
rd = "y ~ 1 + x + C(Group) + C(Group) * x"
model = smf.ols(rd, df_bin).fit(cov_type='HC1')
print(model.summary())

# model 3: result (graph omitted, supplied by STATA)
rd = "y ~ 1 + x + x2 + C(Group) + C(Group)*x + C(Group)*x2"
model = smf.ols(rd, df_bin).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     474.8
Date:                Sat, 22 Jan 2022   Prob (F-statistic):          9.55e-113
Time:                        21:59:22   Log-Likelihood:                 146.84
No. Observations:                 300   AIC:                            -285.7
Df Residuals:                     296   BIC:                            -270.9
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 